# Extension - RRT: basic finding example
* Path finding is using sample-based (RRT) methods rather than constrained optimization to find a collision free path
* Path finding is much easier if a final robot pose is given. We here use IK (formulated as KOMO problem) to first compute a final configuration 'qT'. (Path optimization does this jointly with optimizing the path.)
* Then we can pass the current state and qT to a bi-directional RRT to find a collision free path.
* Note that BotOp (the interface to sim/real) is opened only after we computed the motion. We simply pass the motion to be played by the sim/real.

In [1]:
import robotic as ry
import time

first a minimalistic example for testing:

In [ ]:

C = ry.Config()
C.addFrame("base") .setPosition([0,0,.5])

C.addFrame("ego", "base") \
    .setJoint(ry.JT.transXYPhi, [-1.,1.,-1.,1.,-3.,3.]) \
    .setRelativePosition([.2, .0, .0]) \
    .setShape(ry.ST.ssBox, size=[.05, .3, .05, .01]) \
    .setColor([0, 1., 1.]) \
    .setContact(1)

C.addFrame("obstacle") \
    .setPosition([.0, .0, .5]) \
    .setShape(ry.ST.ssBox, size=[.05, .3, .05, .01]) \
    .setColor([1, .5, 0]) \
    .setContact(1)

C.view(False)

0

-- WARNING:RenderData.cpp:glInitialize:105(-1) FreeType Error: Failed to load font 'ubuntu/Ubuntu-L.ttf' error code: 1 -> text rendering disabled


In [3]:

ry.params_clear()
ry.params_add({'rrt/stepsize':.1, 'rrt/verbose': 3}) #verbose=3 makes it very slow, and displays result, and verbose=4 waits keypress..

rrt = ry.PathFinder()
rrt.helloworld()

Hello, World!
